# Plot the smoothed P(s) curve and its derivative (slope)

In [ ]:
# import core packages
import warnings
warnings.filterwarnings("ignore")
from itertools import combinations
# import semi-core packages
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
plt.style.use('seaborn-poster')
import numpy as np
import pandas as pd
# import open2c libraries
import bioframe
import cooler
import cooltools
import cooltools.expected

import seaborn as sns

In [ ]:
clr = cooler.Cooler('PATH/COOL_FILE.cool')

In [ ]:
# Load chromosome arms sizes
arms = pd.read_table('PATH/ARMS.tab')
arms.head()

In [ ]:
regs = bioframe.parse_regions(arms)
regs.head()

In [ ]:
# cvd == contacts-vs-distance
cvd = cooltools.expected.diagsum(
clr=clr,
regions=regs,
transforms={'balanced': lambda p: p['count']*p['weight1']*p['weight2']}
)
#cvd

In [ ]:
# Aggregate diagonals from different genomic regions together.
# Since all three calcuated statistics are additive, they can be aggregated
# easily via sum() function.

cvd_agg = (
cvd
.groupby('diag')
.agg(
{'n_valid':'sum',
'count.sum':'sum',
'balanced.sum':'sum',
})
.reset_index()
)
# Convert indices of diagonals into genomic separation, expressed in basepairs.
cvd_agg['s_bp'] = (
cvd_agg['diag']
* clr.binsize)
# Now we can calculate the average raw interaction counts and normalized contact frequencies.
cvd_agg['count.avg'] = (
cvd_agg['count.sum']
/ cvd_agg['n_valid']
)
cvd_agg['balanced.avg'] = (
cvd_agg['balanced.sum']
/ cvd_agg['n_valid']
)

In [ ]:
# Plot the P(s) curve

f, ax = plt.subplots(1,1)
ax.loglog(
cvd_agg['s_bp'],
cvd_agg['balanced.avg'],
)

ax.set(
xlabel='separation, bp',
ylabel='IC contact frequency')
ax.set_aspect(1.0)
ax.grid(lw=0.5)

binsize=clr.binsize
# save to pdf
#plt.savefig('{}kb_Ps_curve_arms.pdf'.format(binsize//1000), bbox_inches='tight')

In [ ]:
#Smooth the P(s) curve with logarithmic binning.

# Logbin-expected aggregates P(s) curves per region over exponentially increasing distance bins.
lb_cvd, lb_slopes, lb_distbins = cooltools.expected.logbin_expected(cvd)
# The resulting table contains P(s) curves for each individual region.
# Aggregating these curves into a single genome-wide curve is involving too,
# so we created a separate function for this too.
lb_cvd_agg, lb_slopes_agg = cooltools.expected.combine_binned_expected(
lb_cvd,
binned_exp_slope=lb_slopes
)
lb_cvd_agg['s_bp'] = lb_cvd_agg['diag.avg'] * clr.binsize
lb_slopes_agg['s_bp'] = lb_slopes_agg['diag.avg'] * clr.binsize

In [ ]:
# Plot the smoothed P(s) curve and its derivative

f, axs = plt.subplots(
figsize=(6.5,13),
nrows=2,
gridspec_kw={'height_ratios':[6,2]},
sharex=True)
ax = axs[0]
ax.loglog(
lb_cvd_agg['s_bp'],
lb_cvd_agg['balanced.avg'],
'o-',
markersize=5,
)
ax.set(
ylabel='IC contact frequency',
xlim=(1e3,1e8)
)
ax.set_aspect(1.0)
ax.grid(lw=0.5)
ax = axs[1]
ax.semilogx(
lb_slopes_agg['s_bp'],
lb_slopes_agg['slope'],
alpha=0.5
)
ax.set(
xlabel='separation, bp',
ylabel='slope')
ax.grid(lw=0.5)

#plt.savefig('{}kb_Ps_smooth_curve_derivative_arms.pdf'.format(binsize//1000), bbox_inches='tight')